In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [22]:
# Загрузка даних
data = pd.read_csv('GlobalYouTubeStatistics.csv', delimiter=',', encoding='latin1')

# ВИдалення нульових значень
data = data.dropna()

# Перетворення категоріальних даних на float
label_encoder = LabelEncoder()
data['uploads'] = label_encoder.fit_transform(data['uploads'])
data['uploads'] = data['uploads'].astype(float)
data['Country'] = label_encoder.fit_transform(data['Country'])
data['Country'] = data['Country'].astype(float)
data['channel_type'] = label_encoder.fit_transform(data['channel_type'])
data['channel_type'] = data['channel_type'].astype(float)

# Розбиття даних
X = data[['uploads', 'Country']]
y = data['channel_type']

# Розділення на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Створення першого контейнера для SVM моделі

svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Нормалізація даних
    ('pca', PCA(n_components=2)),   # Зменшення розмірності з PCA
    ('svm', SVC(kernel='linear', C=1.0, random_state=42))  # SVM модель
])

# Тренування моделі та передбачення
svm_pipeline.fit(X_train, y_train)
y_pred = svm_pipeline.predict(X_test)

# Точність моделі
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність моделі SVM: {accuracy}')

Точність моделі SVM: 0.34234234234234234


In [24]:
# Сітка параметрів для SVM моделі
param_grid = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__kernel': ['linear', 'rbf', 'poly']
}

# Використання GridSearchCV для пошуку найкращих параметрів
grid_search = GridSearchCV(svm_pipeline, param_grid, cv=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Вивід найкращих параметрів
print("Найкращі параметри:", grid_search.best_params_)

# Передбачення з використанням найкращої моделі
y_pred = grid_search.predict(X_test)

# Точність оптимізованої моделі
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність оптимізованої моделі SVM: {accuracy}')

Найкращі параметри: {'svm__C': 100, 'svm__kernel': 'poly'}
Точність оптимізованої моделі SVM: 0.3783783783783784


In [25]:
# Створення контейнеру для KNN моделі

knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Нормалізація даних
    ('knn', KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski'))  # KNN модель
])

# Тренування моделі та передбачення
knn_pipeline.fit(X_train, y_train)
y_pred = knn_pipeline.predict(X_test)

# Оцінка точності моделі
accuracy = accuracy_score(y_test, y_pred)
print("Точність моделі KNN:", accuracy)

Точність моделі KNN: 0.38738738738738737


In [26]:
# Сітка параметрів для KNN моделі
param_grid_knn = {
    'knn__n_neighbors': [3, 5, 7],
    'knn__p': [1, 2],
    'knn__metric': ['minkowski', 'euclidean', 'manhattan']
}

# Використання GridSearchCV для пошуку найкращих параметрів
grid_search_knn = GridSearchCV(knn_pipeline, param_grid_knn, cv=2, scoring='accuracy')
grid_search_knn.fit(X_train, y_train)

# Вивід найкращих параметрів
print("Найкращі параметри для KNN:", grid_search_knn.best_params_)

# Передбачення
y_pred = grid_search_knn.predict(X_test)

# Точність
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність оптимізованої моделі KNN: {accuracy}')

Найкращі параметри для KNN: {'knn__metric': 'minkowski', 'knn__n_neighbors': 7, 'knn__p': 1}
Точність оптимізованої моделі KNN: 0.43243243243243246


In [27]:
# Створення контейнера для Decision Tree моделі

dt_pipeline = Pipeline([
    ('minmax_scaler', MinMaxScaler()),  # MinMax масштабування даних
    ('standard_scaler', StandardScaler()),  # Нормалізація даних
    ('dt', DecisionTreeClassifier(random_state=42))  # Decision Tree модель
])

# Тренування моделі на тренувальних даних
dt_pipeline.fit(X_train, y_train)

# Передбачення
y_pred = dt_pipeline.predict(X_test)

# Оцінка точності моделі
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність моделі Decision Tree: {accuracy}')

Точність моделі Decision Tree: 0.2882882882882883


In [28]:
# Сітка параметрів для Decision Tree моделі
param_grid_dt = {
    'dt__criterion': ['gini', 'entropy'],
    'dt__max_depth': [None, 10, 20, 30],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4]
}

# Використання GridSearchCV для пошуку найкращих параметрів
grid_search_dt = GridSearchCV(dt_pipeline, param_grid_dt, cv=2, scoring='accuracy')
grid_search_dt.fit(X_train, y_train)

# Вивід найкращих параметрів
print("Найкращі параметри для Decision Tree:", grid_search_dt.best_params_)

# Передбачення
y_pred = grid_search_dt.predict(X_test)

# Оцінюємо точність оптимізованої Decision Tree моделі на тестовому наборі
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність оптимізованої моделі Decision Tree: {accuracy}')

Найкращі параметри для Decision Tree: {'dt__criterion': 'gini', 'dt__max_depth': 10, 'dt__min_samples_leaf': 4, 'dt__min_samples_split': 2}
Точність оптимізованої моделі Decision Tree: 0.35135135135135137


In [29]:
# Створення контейнеру для моделі RandomForest

rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Нормалізація даних
    ('pca', PCA(n_components=2)),   # Зменшення розмірності з PCA
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))  # Random Forest модель
])

# Тренування моделі на тренувальних даних
rf_pipeline.fit(X_train, y_train)

# Передбачення
y_pred = rf_pipeline.predict(X_test)

# Оцінка точності моделі
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність моделі Random Forest: {accuracy}')

Точність моделі Random Forest: 0.34234234234234234


In [21]:
# Сітка параметрів для Random Forest моделі
param_grid_rf = {
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [None, 10, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

# GridSearchCV для пошуку найкращих параметрів
grid_search_rf = GridSearchCV(rf_pipeline, param_grid_rf, cv=2, scoring='accuracy')
grid_search_rf.fit(X_train, y_train)

# Вивід найкращих параметрів
print("Найкращі параметри для Random Forest:", grid_search_rf.best_params_)

# Передбачення
y_pred = grid_search_rf.predict(X_test)

# Оцінюємо точність оптимізованої Random Forest моделі на тестовому наборі
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність оптимізованої моделі RandomForest: {accuracy}')

Найкращі параметри для Random Forest: {'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 10, 'rf__n_estimators': 50}
Точність оптимізованої моделі RandomForest: 0.40540540540540543
